# Challenge 1: Graph Construction for EEG Channel Relationships

This notebook implements **two graph construction methods** for modeling relationships between EEG channels using Graph Neural Networks (GNN).

## Overview

**Goal**: Construct meaningful graph structures that capture relationships between EEG channels for use in GNN-based models for response time prediction.

**Data Structure**:
- Input: `(batch_size, 129, 200)` - 128 EEG channels + 1 reference channel, 2 seconds @ 100Hz
- Target: Response time in seconds (regression task)

## Graph Construction Methods

We'll implement two approaches:

1. **Functional Connectivity Graph** (Data-driven)
   - Computes correlation between channel time-series
   - Captures actual signal relationships from the data
   - Interpretable and based on real functional connections

2. **Fully Connected Graph with Learnable Weights** (Model-driven)
   - All channels connected to all others
   - Edge weights learned during training
   - Maximum flexibility, lets the model discover important connections


In [4]:
# Import required libraries
import numpy as np
import torch
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

print("✅ Libraries imported successfully")


✅ Libraries imported successfully


## 1. Load Sample Data

First, we'll load a small sample of EEG data to test our graph construction methods. This will help us understand the data structure and build graphs that capture meaningful channel relationships.


In [5]:
# Load data using EEGChallengeDataset
from eegdash.dataset import EEGChallengeDataset
from braindecode.preprocessing import preprocess, Preprocessor, create_windows_from_events
from eegdash.hbn.windows import (
    annotate_trials_with_target,
    add_aux_anchors,
    keep_only_recordings_with,
)

# Configuration
RELEASE_ID = 1  # Using release 1 from merged data
RELEASE_DIR = Path("data_merged") / f"release_{RELEASE_ID}"
EPOCH_LEN_S = 2.0
SFREQ = 100

print(f"📁 Loading data from: {RELEASE_DIR}")

# Load dataset
dataset_ccd = EEGChallengeDataset(
    task="contrastChangeDetection",
    release=f"R{RELEASE_ID}",
    cache_dir=RELEASE_DIR,
    mini=False,
    download=False
)

print(f"✅ Loaded {len(dataset_ccd.datasets)} recordings")

# Preprocess
transformation_offline = [
    Preprocessor(
        annotate_trials_with_target,
        target_field="rt_from_stimulus",
        epoch_length=EPOCH_LEN_S,
        require_stimulus=True,
        require_response=True,
        apply_on_array=False,
    ),
    Preprocessor(add_aux_anchors, apply_on_array=False),
]
preprocess(dataset_ccd, transformation_offline, n_jobs=1)

# Create windows
ANCHOR = "stimulus_anchor"
SHIFT_AFTER_STIM = 0.5
WINDOW_LEN = 2.0

dataset = keep_only_recordings_with(ANCHOR, dataset_ccd)
single_windows = create_windows_from_events(
    dataset,
    mapping={ANCHOR: 0},
    trial_start_offset_samples=int(SHIFT_AFTER_STIM * SFREQ),
    trial_stop_offset_samples=int((SHIFT_AFTER_STIM + WINDOW_LEN) * SFREQ),
    window_size_samples=int(EPOCH_LEN_S * SFREQ),
    window_stride_samples=SFREQ,
    preload=True,
)

print(f"✅ Created {len(single_windows)} windows")

# Extract sample data for graph construction
sample_data = []
sample_targets = []
for i in range(min(100, len(single_windows))):  # Use first 100 windows
    window = single_windows[i]
    X, y = window[0], window[1]
    if isinstance(X, torch.Tensor):
        X = X.numpy()
    sample_data.append(X)
    sample_targets.append(y)

sample_data = np.array(sample_data)  # (100, 129, 200)
sample_targets = np.array(sample_targets)  # (100,)

print(f"\n📊 Sample data shape: {sample_data.shape}")
print(f"   Channels: {sample_data.shape[1]} (128 EEG + 1 reference)")
print(f"   Time points: {sample_data.shape[2]} (2 seconds @ 100Hz)")
print(f"   Target range: {sample_targets.min():.2f} - {sample_targets.max():.2f} seconds")


KeyboardInterrupt: 

## 2. Graph Construction Methods

We'll implement two graph construction approaches that are well-suited for EEG data where we don't have spatial channel information.


### Method 1: Functional Connectivity Graph

**Concept**: Channels that have correlated signals are likely functionally related. We compute pairwise correlations between all channel time-series and create edges for channels with strong correlations.

**Advantages**:
- **Data-driven**: Based on actual signal relationships in your data
- **Interpretable**: You can see which channels are functionally connected
- **Sparse**: Only connects channels with meaningful relationships (reduces noise)

**Steps**:
1. Extract time-series for each of the 128 EEG channels
2. Compute Pearson correlation matrix across all channel pairs
3. Threshold correlations (e.g., |correlation| > 0.3) to create edges
4. Use correlation values as edge weights


In [ ]:
def build_functional_connectivity_graph(eeg_data, threshold=0.3):
    """
    Build graph from functional connectivity (correlation) between channels.
    
    This function computes pairwise correlations between all EEG channels and creates
    edges for channel pairs with correlation above the threshold.
    
    Args:
        eeg_data: (batch, 129, 200) - EEG data (uses first 128 channels, excludes reference)
        threshold: Minimum absolute correlation to create an edge (default: 0.3)
    
    Returns:
        edge_index: (2, num_edges) - Graph edges as source-target pairs
        edge_weights: (num_edges,) - Edge weights (correlation values)
        corr_matrix: (128, 128) - Full correlation matrix for visualization
    """
    # Use only EEG channels (exclude reference channel at index 128)
    eeg_channels = eeg_data[:, :128, :]  # (batch, 128, 200)
    
    # Reshape to (batch*200, 128) - concatenate all time points across all samples
    # This gives us a long time-series for each channel to compute correlations
    signals = eeg_channels.transpose(0, 2, 1).reshape(-1, 128)
    
    # Compute Pearson correlation matrix
    corr_matrix = np.corrcoef(signals.T)  # (128, 128)
    
    # Create adjacency matrix: connect channels with |correlation| > threshold
    adj_matrix = np.abs(corr_matrix) > threshold
    np.fill_diagonal(adj_matrix, False)  # Remove self-loops
    
    # Convert adjacency matrix to edge list format (PyTorch Geometric style)
    edge_index = np.array(np.where(adj_matrix))  # (2, num_edges)
    
    # Edge weights = correlation values (can be positive or negative)
    edge_weights = corr_matrix[adj_matrix]
    
    # Print statistics
    num_edges = edge_index.shape[1]
    num_possible = 128 * 127 // 2  # Maximum edges in undirected graph
    
    print(f"📊 Functional Connectivity Graph:")
    print(f"   Correlation threshold: {threshold}")
    print(f"   Number of edges: {num_edges:,} / {num_possible:,} possible")
    print(f"   Edge density: {num_edges / num_possible:.2%}")
    print(f"   Mean |correlation|: {np.abs(edge_weights).mean():.3f}")
    print(f"   Max |correlation|: {np.abs(edge_weights).max():.3f}")
    
    return edge_index, edge_weights, corr_matrix

# Build functional connectivity graph
print("Building functional connectivity graph...")
fc_edge_index, fc_edge_weights, fc_corr_matrix = build_functional_connectivity_graph(
    sample_data, 
    threshold=0.3
)

print(f"\n✅ Graph created!")
print(f"   Edge index shape: {fc_edge_index.shape}")
print(f"   Edge weights shape: {fc_edge_weights.shape}")


In [ ]:
# Visualize functional connectivity graph
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Full correlation matrix (shows all correlation values)
im1 = axes[0].imshow(fc_corr_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
axes[0].set_title('Correlation Matrix\n(All channel pairs)', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Channel Index')
axes[0].set_ylabel('Channel Index')
plt.colorbar(im1, ax=axes[0], label='Correlation')

# Right: Binary adjacency matrix (shows which channels are connected)
adj_matrix = np.abs(fc_corr_matrix) > 0.3
np.fill_diagonal(adj_matrix, False)
im2 = axes[1].imshow(adj_matrix.astype(int), cmap='Blues', vmin=0, vmax=1)
axes[1].set_title('Graph Adjacency Matrix\n(Connected channels)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Channel Index')
axes[1].set_ylabel('Channel Index')
plt.colorbar(im2, ax=axes[1], label='Connected (1) / Not Connected (0)')

plt.tight_layout()
plt.show()

print("📈 Visualization:")
print("   Left: Correlation values between all channel pairs (red=negative, blue=positive)")
print("   Right: Binary adjacency showing which channels are connected (threshold=0.3)")


### Method 2: Fully Connected Graph with Learnable Weights

**Concept**: Connect all channels to all other channels, and let the model learn which connections are important during training. This is typically implemented using Graph Attention Networks (GAT) or learnable adjacency matrices.

**Advantages**:
- **Maximum flexibility**: Model can discover any channel relationships
- **No assumptions**: Doesn't require thresholding or prior knowledge
- **Adaptive**: Learns task-specific connections

**Trade-offs**:
- **More parameters**: Fully connected means more edges to learn
- **Less interpretable**: Learned weights may not have clear meaning
- **Computationally expensive**: More edges = more computation

**Implementation**: We'll create a fully connected graph structure. The actual learning of edge weights happens during model training (e.g., using GAT attention mechanisms).


In [ ]:
def build_fully_connected_graph(n_channels=128, init_method='small_random'):
    """
    Build fully connected graph (all channels connected to all others).
    
    This creates the graph structure. The actual edge weights will be learned
    during training using mechanisms like Graph Attention Networks (GAT).
    
    Args:
        n_channels: Number of channels (128)
        init_method: How to initialize edge weights
            - 'small_random': Small random values (good for learnable weights)
            - 'uniform': Uniform weights (normalized)
            - 'ones': All weights = 1.0
    
    Returns:
        edge_index: (2, num_edges) - Graph edges (fully connected, no self-loops)
        edge_weights: (num_edges,) - Initial edge weights (will be learned)
        adj_matrix: (n_channels, n_channels) - Adjacency matrix for visualization
    """
    # Create fully connected adjacency (all channels connected, except self-loops)
    adj_matrix = np.ones((n_channels, n_channels), dtype=bool)
    np.fill_diagonal(adj_matrix, False)  # Remove self-loops
    
    # Convert to edge list format
    edge_index = np.array(np.where(adj_matrix))  # (2, num_edges)
    
    # Initialize edge weights (these will be learned/updated during training)
    num_edges = edge_index.shape[1]
    
    if init_method == 'small_random':
        # Small random values - good starting point for learnable weights
        edge_weights = np.random.randn(num_edges) * 0.01
    elif init_method == 'uniform':
        # Uniform weights (normalized)
        edge_weights = np.ones(num_edges) / num_edges
    elif init_method == 'ones':
        # All weights = 1.0
        edge_weights = np.ones(num_edges)
    else:
        raise ValueError(f"Unknown init_method: {init_method}")
    
    # Print statistics
    num_possible = n_channels * (n_channels - 1) // 2
    
    print(f"📊 Fully Connected Graph:")
    print(f"   Initialization: {init_method}")
    print(f"   Number of edges: {num_edges:,} / {num_possible:,} possible")
    print(f"   Edge density: {num_edges / num_possible:.2%} (fully connected)")
    print(f"   Mean edge weight: {edge_weights.mean():.6f}")
    print(f"   Std edge weight: {edge_weights.std():.6f}")
    print(f"\n   Note: Edge weights will be learned during model training")
    print(f"   (e.g., using Graph Attention Network attention mechanisms)")
    
    return edge_index, edge_weights, adj_matrix

# Build fully connected graph
print("Building fully connected graph...")
fc_full_edge_index, fc_full_edge_weights, fc_full_adj_matrix = build_fully_connected_graph(
    n_channels=128,
    init_method='small_random'
)

print(f"\n✅ Graph created!")
print(f"   Edge index shape: {fc_full_edge_index.shape}")
print(f"   Edge weights shape: {fc_full_edge_weights.shape}")


In [ ]:
# Visualize fully connected graph
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Left: Binary adjacency matrix (should be all ones except diagonal)
im1 = axes[0].imshow(fc_full_adj_matrix.astype(int), cmap='Blues', vmin=0, vmax=1)
axes[0].set_title('Fully Connected Adjacency Matrix\n(All channels connected)', 
                  fontsize=12, fontweight='bold')
axes[0].set_xlabel('Channel Index')
axes[0].set_ylabel('Channel Index')
plt.colorbar(im1, ax=axes[0], label='Connected (1) / Not Connected (0)')

# Right: Initial edge weights (reshaped to matrix for visualization)
weight_matrix = np.zeros((128, 128))
weight_matrix[fc_full_edge_index[0], fc_full_edge_index[1]] = fc_full_edge_weights
im2 = axes[1].imshow(weight_matrix, cmap='viridis')
axes[1].set_title('Initial Edge Weights\n(Will be learned during training)', 
                  fontsize=12, fontweight='bold')
axes[1].set_xlabel('Channel Index')
axes[1].set_ylabel('Channel Index')
plt.colorbar(im2, ax=axes[1], label='Initial Weight')

plt.tight_layout()
plt.show()

print("📈 Visualization:")
print("   Left: Binary adjacency showing all channels are connected (except self-loops)")
print("   Right: Initial edge weights (small random values that will be learned)")


## 3. Comparison of Graph Construction Methods

Let's compare the two methods to understand their characteristics:


In [ ]:
# Compare the two graph construction methods
import pandas as pd

comparison_data = {
    'Method': [
        'Functional Connectivity',
        'Fully Connected (Learnable)'
    ],
    'Number of Edges': [
        fc_edge_index.shape[1],
        fc_full_edge_index.shape[1]
    ],
    'Edge Density': [
        f"{fc_edge_index.shape[1] / (128 * 127 // 2):.2%}",
        f"{fc_full_edge_index.shape[1] / (128 * 127 // 2):.2%}"
    ],
    'Edge Weight Source': [
        'Correlation values (from data)',
        'Learnable (initialized, then trained)'
    ],
    'Interpretability': [
        'High - shows actual signal relationships',
        'Low - learned weights may not be interpretable'
    ],
    'Flexibility': [
        'Medium - fixed by data correlations',
        'High - model learns optimal connections'
    ],
    'Computational Cost': [
        'Lower - sparse graph',
        'Higher - fully connected'
    ],
    'Best For': [
        'Understanding channel relationships, interpretable models',
        'Maximum model flexibility, task-specific learning'
    ]
}

comparison_df = pd.DataFrame(comparison_data)
print("📊 Graph Construction Methods Comparison:\n")
print(comparison_df.to_string(index=False))

print("\n" + "="*90)
print("Summary:")
print("="*90)
print("1. Functional Connectivity: Data-driven, interpretable, sparse")
print("   → Use when you want to understand channel relationships")
print("   → Good starting point for exploration")
print()
print("2. Fully Connected (Learnable): Maximum flexibility, task-adaptive")
print("   → Use when you want the model to discover optimal connections")
print("   → Requires more parameters and computation")
print()
print("💡 Recommendation: Try both and compare performance!")
print("   Start with functional connectivity for interpretability,")
print("   then try fully connected if you need better performance.")


In [ ]:
## 4. Convert to PyTorch Format

For use with PyTorch Geometric or other GNN libraries, we need to convert the graphs to PyTorch tensors.


In [ ]:
def convert_to_pytorch(edge_index, edge_weights=None):
    """
    Convert numpy arrays to PyTorch tensors for use with PyTorch Geometric.
    
    Args:
        edge_index: (2, num_edges) numpy array - source and target node indices
        edge_weights: (num_edges,) numpy array (optional) - edge weights
    
    Returns:
        edge_index_tensor: torch.LongTensor - edge connections
        edge_weights_tensor: torch.FloatTensor (if provided) - edge weights
    """
    edge_index_tensor = torch.from_numpy(edge_index).long()
    
    if edge_weights is not None:
        edge_weights_tensor = torch.from_numpy(edge_weights).float()
        return edge_index_tensor, edge_weights_tensor
    else:
        return edge_index_tensor

# Convert both graphs to PyTorch tensors
print("Converting graphs to PyTorch format...")
fc_edge_index_torch, fc_edge_weights_torch = convert_to_pytorch(fc_edge_index, fc_edge_weights)
fc_full_edge_index_torch, fc_full_edge_weights_torch = convert_to_pytorch(
    fc_full_edge_index, fc_full_edge_weights
)

print("✅ Graphs converted to PyTorch tensors:")
print(f"   Functional Connectivity:")
print(f"      Edge index: {fc_edge_index_torch.shape}")
print(f"      Edge weights: {fc_edge_weights_torch.shape}")
print(f"   Fully Connected:")
print(f"      Edge index: {fc_full_edge_index_torch.shape}")
print(f"      Edge weights: {fc_full_edge_weights_torch.shape}")

# Store graphs in a dictionary for easy access
graphs = {
    'functional': {
        'edge_index': fc_edge_index_torch,
        'edge_weights': fc_edge_weights_torch,
        'description': 'Functional connectivity (correlation-based, sparse)'
    },
    'fully_connected': {
        'edge_index': fc_full_edge_index_torch,
        'edge_weights': fc_full_edge_weights_torch,
        'description': 'Fully connected (learnable weights, dense)'
    }
}

print(f"\n✅ Graphs stored in dictionary 'graphs' for use in model")
print(f"   Access with: graphs['functional'] or graphs['fully_connected']")


## 5. Next Steps

Now that we have two graph construction methods ready, you can use them in your GNN model:

### Using the Graphs

Both graphs are stored in the `graphs` dictionary with the following structure:
- `graphs['functional']` - Functional connectivity graph (sparse, data-driven)
- `graphs['fully_connected']` - Fully connected graph (dense, learnable)

Each graph contains:
- `edge_index`: `(2, num_edges)` tensor - edge connections (source → target)
- `edge_weights`: `(num_edges,)` tensor - edge weights

### Example Usage

```python
# Get functional connectivity graph
func_graph = graphs['functional']
edge_index = func_graph['edge_index']  # (2, num_edges)
edge_weights = func_graph['edge_weights']  # (num_edges,)

# Use with PyTorch Geometric
from torch_geometric.nn import GCNConv, GATConv

# Example: Graph Convolutional Network
class EEGGNN(nn.Module):
    def __init__(self, num_channels=128, hidden_dim=64):
        super().__init__()
        self.conv1 = GCNConv(num_channels, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        # ... rest of model
```

### Recommendations

1. **Start with functional connectivity**: It's interpretable and based on real data relationships
2. **Try fully connected if needed**: If functional connectivity doesn't perform well, try the learnable approach
3. **Combine with temporal models**: Use GNN for spatial relationships + GRU/LSTM for temporal patterns
4. **Experiment with thresholds**: For functional connectivity, try different correlation thresholds (0.2, 0.3, 0.4)
